In [1]:
from pathlib import Path
from dotenv import load_dotenv

# import os
#
# os.environ["OPENAI_API_KEY"] = "..."
# os.environ["PAREA_API_KEY"] = "..."

load_dotenv(Path("../.env.test"), override=True)

True

In [2]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from openai import OpenAI
from parea import Parea, trace

parea = Parea()
openai = OpenAI()

parea.wrap_openai_client(openai)

In [4]:
from parea.schemas import Log, EvaluationResult


def score_answer(log: Log) -> EvaluationResult:
    if log.target:
        output = log.output.split("#### ")[-1]
        target = log.target.split("#### ")[-1]
        return EvaluationResult("correctness", int(output == target))


@trace(eval_funcs=[score_answer])
def langchain_chain(inputs):
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    messages = [
        (
            "system",
            "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples.",
        ),
        ("user", "{question}")
    ]

    chain = (
        ChatPromptTemplate.from_messages(messages)
        | ChatOpenAI(model="gpt-3.5-turbo")
        | StrOutputParser()
    )

    answer = chain.invoke(inputs)
    return answer


In [7]:
experiment_kwargs = dict(
    data="gsm8k-testset",
    n_workers=2,
)

parea.experiment(
    name="wibbly-wobbly",
    func=langchain_chain,
    **experiment_kwargs
).run()

Run name set to: leary-plug, since a name was not provided.
Fetching test collection: gsm8k-testset
Fetched 5 test cases from collection: gsm8k-testset 



100%|██████████| 5/5 [00:01<00:00,  2.63it/s]
0it [00:04, ?it/s]


Experiment wibbly-wobbly Run leary-plug stats:
{
  "latency": "0.73",
  "input_tokens": "0.00",
  "output_tokens": "0.00",
  "total_tokens": "0.00",
  "cost": "0.00000",
  "correctness": "0.40"
}


View experiment & traces at: https://app.parea.ai/experiments/wibbly-wobbly/38aa90aa-c266-448a-abad-c659db606ba8



In [15]:
from zenbase.types import LMRequest, deflm

# Step 1: Add the deflm decorator
# Step 2: Incorporate few-shot demonstrations

@deflm # Step 1
@trace(eval_funcs=[score_answer])
def zen_chain(request: LMRequest):
    from langchain_openai import ChatOpenAI
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    messages = [
        (
            "system",
            "You are an expert math solver. Your answer must be just the number with no separators, and nothing else. Follow the format of the examples.",
        )
    ]

    # Step 2
    for demo in request.zenbase.demos:
        messages += [
            ("user", demo.inputs["question"]),
            ("assistant", demo.outputs["target"]),
        ]

    messages.append(("user", "{question}"))

    chain = (
        ChatPromptTemplate.from_messages(messages)
        | ChatOpenAI(model="gpt-3.5-turbo")
        | StrOutputParser()
    )

    answer = chain.invoke(request.inputs)
    return answer


In [10]:
from zenbase.optim.metric.labeled_few_shot import LabeledFewShot
from zenbase.helpers.parea import ZenParea

optimizer = LabeledFewShot(
    demoset=ZenParea.test_collection_demos(parea.get_collection("gsm8k-demoset")),
    shots=3
)

best_fn, candidate_results = optimizer.perform(
    zen_chain,
    evaluator=ZenParea.metric_evaluator(**experiment_kwargs),
    samples=2,
    rounds=1,
)

Run name set to: still-tule, since a name was not provided.
Fetching test collection: gsm8k-testset
Fetched 5 test cases from collection: gsm8k-testset 



100%|██████████| 5/5 [00:05<00:00,  1.07s/it]
0it [00:04, ?it/s]


Experiment zenbase-fully-configurable-bandwidth-monitored-strategy Run still-tule stats:
{
  "latency": "1.77",
  "input_tokens": "0.00",
  "output_tokens": "0.00",
  "total_tokens": "0.00",
  "cost": "0.00000",
  "correctness": "0.60"
}


View experiment & traces at: https://app.parea.ai/experiments/zenbase-fully-configurable-bandwidth-monitored-strategy/123e90c5-8ba5-41dc-81af-c20178ce9461

Run name set to: tubby-fils, since a name was not provided.
Fetching test collection: gsm8k-testset
Fetched 5 test cases from collection: gsm8k-testset 



100%|██████████| 5/5 [00:03<00:00,  1.31it/s]
0it [00:04, ?it/s]


Experiment zenbase-profit-focused-disintermediate-challenge Run tubby-fils stats:
{
  "latency": "1.25",
  "input_tokens": "0.00",
  "output_tokens": "0.00",
  "total_tokens": "0.00",
  "cost": "0.00000",
  "correctness": "0.60"
}


View experiment & traces at: https://app.parea.ai/experiments/zenbase-profit-focused-disintermediate-challenge/fa550de9-3bcd-43bf-808a-92e997a3e845



In [12]:
output = best_fn({"question": "What is 2+2?"})
output

'4'

In [13]:
# You can even run your function asynchronously in a coroutine
%autoawait

await best_fn.coroutine({
    "question": "I have 30% of company and Mamad has 40% of company and there are 10M shares, how many shares are unassigned?"
})

IPython autoawait is `on`, and set to use `asyncio`


'The total percentage of shares assigned is 30% + 40% = 70%.\nThis means that 100% - 70% = 30% of shares are unassigned.\nAs there are 10M shares in total, the number of unassigned shares is 30% of 10M, which is 0.3 * 10M = 3M.\n#### 3000000'

In [16]:
# You can also save the zenbase params for re-use
import pickle

pickled_zenbase = pickle.dumps(best_fn.zenbase)
zen_chain.zenbase = pickle.loads(pickled_zenbase)

zen_chain({"question": "What is 2 + 2?"}) # uses the best few-shot demos

'4'